In [1]:
import os 

In [2]:
os.chdir('/home/yehudan/Desktop/DecisionTree/DecisionTree')

In [1]:
import time
from datasets import mse_comparison 
from scipy.io.arff import loadarff
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('datasets/Pakistan_Largest_Ecommerce_Dataset.csv')

/data/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
def move_columns_to_end(df,col1,col2=None):
    cols = list(df.columns.values) #Make a list of all of the columns in the df
    cols.pop(cols.index(col1)) #Remove col1 from list
    if col2:
        cols.pop(cols.index(col2)) #Remove col2 from list
    if col2:
        df = df[cols+[col1,col2]] #Create new dataframe with columns in the order you want
    else:
        df = df[cols+[col1]]
    return df

In [6]:
def preprocessing(df):
    df1= df.copy()
    df1['Mobiles & Tablets'] = df['Mobiles & Tablets'].apply(lambda x: 0 if x<=df['Mobiles & Tablets'].median() else 1)
    df1['Beauty & Grooming'] = df['Beauty & Grooming'].apply(lambda x: 0 if x<=df['Beauty & Grooming'].median() else 1)
    return df1

In [7]:
pakistan_columns = ['grand_total', 'category_name_1', 'payment_method', 'Customer ID']
pakistan_filtered = df[pakistan_columns]
grouped_data = pakistan_filtered.groupby(by=['category_name_1', 'Customer ID'], as_index=False).agg({'grand_total': 'sum'})
pivoted = grouped_data.pivot(index='Customer ID', columns='category_name_1', values='grand_total')
pivoted = pivoted.reset_index().rename_axis(None, axis=1)
# pivoted = pivoted.fillna(0)
# final_df = pivoted.drop(['Customer ID'], axis='columns')
# final_df.head(2)

In [8]:
pivoted[pivoted.notnull().sum(axis=1)>=9].shape

(1325, 17)

In [9]:
final_df = pivoted[pivoted.notnull().sum(axis=1)>=9]
final_df = final_df.drop(['Customer ID'], axis='columns')
final_df = final_df.fillna(0)
final_df.shape

(1325, 16)

In [10]:
final_df = move_columns_to_end(final_df,'Mobiles & Tablets','Beauty & Grooming')
final_df.head()

,Appliances,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N,Mobiles & Tablets,Beauty & Grooming
3,285592.7,0.0,99.00,593501.20,999.0,22185.0,2250.0,97804.55,1249.00,495.0,32990.0,94530.0,29158.50,115044.85,1257478.35,34219.30
12,207775.3,0.0,3230.00,53644.00,0.0,130560.0,3018.0,27770.00,0.00,720.0,38498.0,0.0,41868.00,12974.00,30886.00,66022.00
14,0.0,1730.0,1205.00,0.00,0.0,12171.0,1320.0,4721.00,125971.93,0.0,1065.0,2234.0,0.00,0.00,1205.00,0.00
15,0.0,6399.0,28345.11,0.00,3041.0,1475.0,15247.0,33108.60,4630.00,1863.0,9127.0,45546.0,1927773.13,994.00,185968.40,54005.22
19,1363251.0,0.0,360.00,1076207.68,13536.2,10230.0,1740.0,110845.40,0.00,0.0,21671.5,328188.0,48709.00,124325.25,1219169.35,37893.00


In [11]:
final_df_classifiction = preprocessing(final_df)
final_df_classifiction.head()

,Appliances,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N,Mobiles & Tablets,Beauty & Grooming
3,285592.7,0.0,99.00,593501.20,999.0,22185.0,2250.0,97804.55,1249.00,495.0,32990.0,94530.0,29158.50,115044.85,1,1
12,207775.3,0.0,3230.00,53644.00,0.0,130560.0,3018.0,27770.00,0.00,720.0,38498.0,0.0,41868.00,12974.00,1,1
14,0.0,1730.0,1205.00,0.00,0.0,12171.0,1320.0,4721.00,125971.93,0.0,1065.0,2234.0,0.00,0.00,0,0
15,0.0,6399.0,28345.11,0.00,3041.0,1475.0,15247.0,33108.60,4630.00,1863.0,9127.0,45546.0,1927773.13,994.00,1,1
19,1363251.0,0.0,360.00,1076207.68,13536.2,10230.0,1740.0,110845.40,0.00,0.0,21671.5,328188.0,48709.00,124325.25,1,1


In [12]:
final_df_classifiction['Beauty & Grooming'].value_counts()

0    663
1    662
Name: Beauty & Grooming, dtype: int64

1325 rows (more than 9 not nulls)

In [13]:
res_Kfold_all, res_Kfold_ind, res_Kfold_loo, res_Kfold_choose = mse_comparison.mse_comparison(path=None,
                                                                    df=final_df_classifiction, n_features=14,
                                                                                              max_depth=np.inf,
                              use_pruning_for_loo=True,
                              use_pruning_2_models=True, n_splits=10,regression=False,
                                  random_state=0)

Cell execution time: 2023-04-04 22:32:18.072055
cc


CV progress bar: 0it [00:00, ?it/s]

for all print:
[0.3834024823318448, 0.3712228824584227, 0.36870539010583314, 0.36701768573538207, 0.36326605956190006, 0.3800446538448015, 0.3833954502303013, 0.39472943989311204, 0.39892584648922325, 0.4014169684610246, 0.3980679301009107, 0.40560985900636404, 0.41022291761893037, 0.41442284026581333, 0.41400267219858644, 0.4165219225765619, 0.41820259484546957, 0.41861924686192464, 0.41987799303821943, 0.421558665307127, 0.4219788333743539, 0.42449808375232945, 0.42449808375232945]
Root
depth=0| Split rule- x_i to split:0, value to split:59208.5
   | Impurity of the node: 595.98
   | Impurity of the best split: 0.46
   | Count of observations in node: 1192
depth=1|-------- Split rule- x_i to split:12, value to split:6877.2
           | Impurity of the node: 434.88
           | Impurity of the best split: 0.47
           | Count of observations in node: 894
depth=2|---------------- Split rule- x_i to split:3, value to split:2114.5
                   | Impurity of the node: 275.14
    

CV progress bar: 1it [50:03, 3003.54s/it]

************************************************************sub tree number 0 ************************************************************
Root
depth=0| Split rule- x_i to split:0, value to split:32066.6
   | Impurity of the node: 297.99
   | Impurity of the best split: 0.44
   | Count of observations in node: 1192
   |LOO score: 287.2255663252763
depth=1|-------- Split rule- x_i to split:3, value to split:1215.0
           | Impurity of the node: 175.64
           | Impurity of the best split: 0.44
           | Count of observations in node: 757
           |LOO score: 172.99930967694263
depth=2|---------------- Split rule- x_i to split:13, value to split:4512.86
                   | Impurity of the node: 96.31
                   | Impurity of the best split: 0.39
                   | Count of observations in node: 471
                   |LOO score: 94.23825559547375
depth=3|------------------------ Split rule- x_i to split:11, value to split:36839.175
                           | Impu

...

************************************************************sub tree number 0 ************************************************************
Root
depth=0| Split rule- x_i to split:3, value to split:60471.575
   | Impurity of the node: 298.1
   | Impurity of the best split: 0.43
   | Count of observations in node: 1193
   |LOO score: 282.70890164282486
depth=1|-------- Split rule- x_i to split:0, value to split:7859.895
           | Impurity of the node: 236.63
           | Impurity of the best split: 0.45
           | Count of observations in node: 984
           |LOO score: 234.02128669094566
depth=2|---------------- Split rule- x_i to split:7, value to split:49041.1
                   | Impurity of the node: 98.77
                   | Impurity of the best split: 0.37
                   | Count of observations in node: 491
                   |LOO score: 96.52608469064708
depth=3   leaf node                               | Count of observations in node: 469
                              

CV progress bar: 9it [6:32:48, 2578.24s/it]

************************************************************sub tree number 0 ************************************************************
Root
depth=0| Split rule- x_i to split:0, value to split:32066.6
   | Impurity of the node: 298.25
   | Impurity of the best split: 0.43
   | Count of observations in node: 1193
   |LOO score: 287.0477011949728
depth=1|-------- Split rule- x_i to split:3, value to split:5168.775
           | Impurity of the node: 175.49
           | Impurity of the best split: 0.44
           | Count of observations in node: 760
           |LOO score: 172.72237822707828
depth=2|---------------- Split rule- x_i to split:11, value to split:36839.175
                   | Impurity of the node: 110.51
                   | Impurity of the best split: 0.4
                   | Count of observations in node: 535
                   |LOO score: 108.14125832154093
depth=3   leaf node                               | Count of observations in node: 471
                            

CV progress bar: 10it [7:20:12, 2641.22s/it]

************************************************************sub tree number 0 ************************************************************
Root
depth=0| Split rule- x_i to split:3, value to split:69552.225
   | Impurity of the node: 298.16
   | Impurity of the best split: 0.43
   | Count of observations in node: 1193
   |LOO score: 288.1340579190819
depth=1|-------- Split rule- x_i to split:0, value to split:8392.75
           | Impurity of the node: 242.48
           | Impurity of the best split: 0.46
           | Count of observations in node: 992
           |LOO score: 232.51490971269666
depth=2|---------------- Split rule- x_i to split:3, value to split:2114.5
                   | Impurity of the node: 107.41
                   | Impurity of the best split: 0.4
                   | Count of observations in node: 509
                   |LOO score: 104.6017877248967
depth=3   leaf node                               | Count of observations in node: 378
                                

In [14]:
import statistics
print(f'for all stdev {statistics.stdev(res_Kfold_all)}')
print(f'for ind stdev {statistics.stdev(res_Kfold_ind)}')
print(f'for loo stdev {statistics.stdev(res_Kfold_loo)}')

for all stdev 0.02598655941375513
for ind stdev 0.0292551255314933
for loo stdev 0.026880427054530656


In [15]:
_, _, _, _ = mse_comparison.mse_comparison(path=None,
                                                                    df=final_df_classifiction, n_features=14,
                                                                max_depth=np.inf,only_choose_one=True,
                                                                    n_splits=10,regression=False,
                                                                  random_state=0)

Cell execution time: 2023-04-05 05:52:30.308891
cc


CV progress bar: 0it [00:00, ?it/s]

starting choose one model



Choose one progress bar: 0it [00:00, ?it/s]

Starting all
[0.3852749402303847, 0.3754683764399044, 0.3679830471636601, 0.3698522060421648, 0.37264073027602695, 0.3731427950445555, 0.38015648772006083, 0.38012605955227124, 0.37875461856118237, 0.38993697022386437, 0.39881330145620525, 0.398335144533797, 0.3964746794175179, 0.3978765485763965, 0.4076592045207564, 0.4002043034123017, 0.40300586828950224, 0.40627037600521626, 0.4090675940013041, 0.40953705716148664, 0.41139752227776566, 0.41093240599869596, 0.4104672897196262, 0.4104672897196262]
runtime for all model:405.53431010246277 seconds
Finish all
Starting ind
[0.37315366224733754, 0.3395479243642686, 0.35350575961747444, 0.33578787220169526, 0.34420778091719195, 0.3451467072375571, 0.34608563355792216, 0.3414127363616605, 0.3469984785916105, 0.3497804825038035, 0.3563051510541187, 0.360043468811128, 0.36844599000217343, 0.3647076722451641, 0.36096066072592914, 0.36190393392740705, 0.3665637904803304, 0.3684372962399478, 0.3684372962399478]
[0.361051945229298, 0.3647859161051

...

runtime for ind model:313.91470551490784 seconds
Finish ind
MSE all: 3.6296568627450982
MSE ind: 0
for ind was chosen


[0.35461130058718043, 0.340329102352238, 0.30512288597447346, 0.3260644843711543, 0.3118350268977884, 0.3327836573960128, 0.3420062585703737, 0.3302485847895643, 0.3327449808375233, 0.3335853169719771, 0.3436658345346507, 0.34367989873773774, 0.3369783059667382, 0.3411694384866918, 0.33529763369783067, 0.3453605710066453, 0.3445202348721915, 0.3637811609999649, 0.36629513730178265, 0.36629513730178265]
[0.3646601736929081, 0.3755318026792307, 0.34454133117682223, 0.3461833268872403, 0.35959354453078307, 0.36461798108364685, 0.37970183889455367, 0.37130550965155934, 0.3830315389754228, 0.3880208150205689, 0.40060476073274504, 0.41316409408951865, 0.4140009141732007, 0.41903941492915153, 0.42156042333251287, 0.4198832671143771, 0.42240427551773846, 0.4190499630814669, 0.4207200871980591, 0.42323757955064867, 0.42323757955064867]


CV progress bar: 10it [16:42:20, 6014.06s/it]

runtime for ind model:393.1891579627991 seconds
runtime for choose one model:5270.757545471191 seconds
MSE for all model:   nan
MSE for ind model:   nan
MSE LOO model:       nan
MSE choose one model:0.3528394850763272
stdev choose one model:0.0292551255314933
Total runtime: 1002.3435271143913 minutes



/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:276: RuntimeWarning: Mean of empty slice.
  print(f'MSE for all model:   {np.array(mse_all).mean()}')
/data/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:277: RuntimeWarning: Mean of empty slice.
  print(f'MSE for ind model:   {np.array(mse_ind).mean()}')
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:278: RuntimeWarning: Mean of empty slice.
  print(f'MSE LOO model:       {np.array(mse_loo).mean()}')


In [14]:
mse_comparison.mse_comparison(path=None,
                                df=final_df_classifiction, n_features=14,
                                only_related_works=True,
                                n_splits=10,regression=False,
                              random_state=0)

Cell execution time: 2023-05-31 11:25:09.721008


CV progress bar: 1it [19:52, 1192.31s/it]

List MSE chains: [0.3609022556390977]
List MSE stacked: [0.3609022556390977]


CV progress bar: 2it [38:02, 1132.28s/it]

List MSE chains: [0.3609022556390977, 0.37969924812030076]
List MSE stacked: [0.3609022556390977, 0.3684210526315789]


...

List MSE chains: [0.3609022556390977, 0.37969924812030076, 0.35714285714285715, 0.3082706766917293, 0.3458646616541353, 0.3939393939393939, 0.3106060606060606, 0.3977272727272727, 0.3484848484848485]
List MSE stacked: [0.3609022556390977, 0.3684210526315789, 0.40977443609022557, 0.3082706766917293, 0.42105263157894735, 0.4583333333333333, 0.4393939393939394, 0.38636363636363635, 0.4431818181818182]


CV progress bar: 10it [2:53:02, 1038.21s/it]

List MSE chains: [0.3609022556390977, 0.37969924812030076, 0.35714285714285715, 0.3082706766917293, 0.3458646616541353, 0.3939393939393939, 0.3106060606060606, 0.3977272727272727, 0.3484848484848485, 0.3371212121212121]
List MSE stacked: [0.3609022556390977, 0.3684210526315789, 0.40977443609022557, 0.3082706766917293, 0.42105263157894735, 0.4583333333333333, 0.4393939393939394, 0.38636363636363635, 0.4431818181818182, 0.3409090909090909]
------------------------------------------------------------------------------------------
MSE chains : 0.3539758487126908
MSE stacked: 0.3936602870813397
stdev choose chains model 0.03087770296992884
stdev choose stacked model 0.04896028004030264
MSE for all model:   nan
MSE for ind model:   nan
MSE LOO model:       nan
Total runtime: 173.03478962183 minutes



/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:305: RuntimeWarning: Mean of empty slice.
  print(f'MSE for all model:   {np.array(mse_all).mean()}')
/data/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:306: RuntimeWarning: Mean of empty slice.
  print(f'MSE for ind model:   {np.array(mse_ind).mean()}')
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:307: RuntimeWarning: Mean of empty slice.
  print(f'MSE LOO model:       {np.array(mse_loo).mean()}')


([], [], [], [])